# K-Means Clustering from Scratch

Run in Colab. This notebook implements K-Means from scratch (no scikit-learn KMeans) and compares with sklearn for sanity. Steps: data gen, init, iterate assign/update, convergence, visualization, metrics.

In [ ]:
# Install (Colab)
!pip install -q matplotlib seaborn scikit-learn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
np.random.seed(42)
sns.set(style='whitegrid')

## Generate synthetic data

In [ ]:
X, y_true = make_blobs(n_samples=500, centers=4, cluster_std=0.60, random_state=42)
plt.scatter(X[:,0], X[:,1], s=10)
plt.title('Synthetic data')
plt.show()

## K-Means from scratch

In [ ]:
def kmeans_from_scratch(X, k=4, max_iters=100, tol=1e-4):
    n, d = X.shape
    # init centroids randomly from data
    idx = np.random.choice(n, k, replace=False)
    centroids = X[idx]
    history = []
    for i in range(max_iters):
        # assign step
        dist = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
        labels = dist.argmin(axis=1)
        # update step
        new_centroids = np.array([X[labels==j].mean(axis=0) for j in range(k)])
        # handle empty clusters by reinit
        for j in range(k):
            if np.isnan(new_centroids[j]).any():
                new_centroids[j] = X[np.random.choice(n)]
        shift = np.linalg.norm(new_centroids - centroids)
        history.append((centroids.copy(), labels.copy(), shift))
        centroids = new_centroids
        if shift < tol:
            break
    return centroids, labels, history

In [ ]:
k = 4
centroids, labels, history = kmeans_from_scratch(X, k=k, max_iters=50)
plt.figure(figsize=(6,5))
for j in range(k):
    pts = X[labels==j]
    plt.scatter(pts[:,0], pts[:,1], s=10, label=f'Cluster {j}')
plt.scatter(centroids[:,0], centroids[:,1], c='black', s=120, marker='X', label='Centroids')
plt.legend()
plt.title('K-Means from scratch result')
plt.show()

## Convergence diagnostics (centroid shift)

In [ ]:
shifts = [h[2] for h in history]
plt.plot(shifts, marker='o')
plt.xlabel('Iteration')
plt.ylabel('Centroid shift (L2)')
plt.title('Convergence')
plt.show()

## Silhouette score (from scratch labels)

In [ ]:
score = silhouette_score(X, labels)
score

## Compare with scikit-learn KMeans

In [ ]:
sk_kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto').fit(X)
sk_labels = sk_kmeans.labels_
sk_score = silhouette_score(X, sk_labels)
sk_score

## Overlay comparison

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
for j in range(k):
    pts = X[labels==j]
    plt.scatter(pts[:,0], pts[:,1], s=10)
plt.scatter(centroids[:,0], centroids[:,1], c='black', s=120, marker='X')
plt.title('Scratch K-Means')

plt.subplot(1,2,2)
for j in range(k):
    pts = X[sk_labels==j]
    plt.scatter(pts[:,0], pts[:,1], s=10)
plt.scatter(sk_kmeans.cluster_centers_[:,0], sk_kmeans.cluster_centers_[:,1], c='black', s=120, marker='X')
plt.title('sklearn KMeans')
plt.tight_layout()
plt.show()

## Notes
- Extend with different inits (k-means++), distance metrics, or elbow/optimal k.
- Add your own dataset (e.g., image features, embeddings).
- Record a short video walkthrough showing code, plots, and metrics.
